In [1]:
import requests
import json
import pandas as pd
from langchain.document_loaders import DataFrameLoader

In [27]:
vectordb_url="http://vectordb.runai-vast.svc.cluster.local/retrieve-chunks/"

query = "What did the iCCT model reveal about the U.S. population's beliefs regarding responsible AI?"

query_string = {'query':query}

response = requests.post(vectordb_url,json=query_string)

#print (response.json()['results'])

results = pd.DataFrame(response.json()['results'])

print (results)

[{'element_id': '975678b31c3e6e7cc3b0d2e7c18cb30a', 'cleaned_text': '2024 4 2 0 2 J 6 ] Y C . [ 1 9 2 0 0 0 . 2 0 4 2 : arXiv. X EXPLORING PUBLIC OPINION ON RESPONSIBLE AI THROUGH THE LENS OF CULTURAL CONSENSUS THEORY Necdet Gürkan Jordan W. Suchow The School of Business Stevens Institute of Technology ngurkan@stevens.edu The School of Business Stevens Institute of Technology jws@stevens.edu ABSTRACT As the societal implications of Artificial Intelligence (AI) continue to grow, the pursuit of responsible AI necessitates public engagement in its development and governance processes. This involvement is crucial for capturing diverse perspectives and promoting equitable practices and outcomes. We applied Cultural Consensus Theory (CCT) to nationally representative survey dataset on various aspects of AI to discern beliefs and attitudes about responsible AI in the United States. Our results offer valuable insights by identifying shared and contrasting views on responsible AI. Furthermore, 

In [28]:
rerank_url="http://rerankqa-mistral-4b.runai-genai.svc.cluster.local/v1/ranking"

start_string = '{"model": "nvidia/nv-rerankqa-mistral-4b-v3","query": {"text":"' + query + '"},"passages": [{"text": "'
 
middle_string = '"},{"text": "'.join(results['cleaned_text'])

middle_string = middle_string.encode("ascii","ignore")

middle_string = middle_string.decode()

middle_string = middle_string.replace('"','')

end_string='"}],"truncate": "END"}'

rerank_string = start_string+middle_string+end_string

#print (re

json_object = json.loads(rerank_string)

#print (json.dumps(json_object, indent=1))

In [29]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

response = requests.post(rerank_url, headers=headers, data=rerank_string)

ranking = pd.DataFrame(response.json()['rankings'])

citing_text = ""

if ranking['logit'].loc[0] > 10:
    citing_text = results['cleaned_text'].loc[ranking['index'].loc[0]]

print(citing_text)

print(ranking)

2024 4 2 0 2 J 6 ] Y C . [ 1 9 2 0 0 0 . 2 0 4 2 : arXiv. X EXPLORING PUBLIC OPINION ON RESPONSIBLE AI THROUGH THE LENS OF CULTURAL CONSENSUS THEORY Necdet Gürkan Jordan W. Suchow The School of Business Stevens Institute of Technology ngurkan@stevens.edu The School of Business Stevens Institute of Technology jws@stevens.edu ABSTRACT As the societal implications of Artificial Intelligence (AI) continue to grow, the pursuit of responsible AI necessitates public engagement in its development and governance processes. This involvement is crucial for capturing diverse perspectives and promoting equitable practices and outcomes. We applied Cultural Consensus Theory (CCT) to nationally representative survey dataset on various aspects of AI to discern beliefs and attitudes about responsible AI in the United States. Our results offer valuable insights by identifying shared and contrasting views on responsible AI. Furthermore, these findings serve as critical reference points for developers and 